In [1]:
import aadc
import aadc.quantlib as ql
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
import random
import time

In [2]:
def createRandomSwaps(num_trades, todays_date, calendar):
    random.seed(42)
    portfolio = []

    for _ in range(num_trades):
        discounting_term_structure = ql.RelinkableYieldTermStructureHandle()
        forecasting_term_structure = ql.RelinkableYieldTermStructureHandle()

        nominal = 1000000.0 * (100.0 + random.randint(0, 399)) * (-1.0 + 2 * random.randint(0, 1))

        # Fixed leg
        fixed_leg_frequency = ql.Bimonthly
        fixed_leg_convention = ql.Unadjusted
        floating_leg_convention = ql.ModifiedFollowing
        fixed_leg_day_counter = ql.Thirty360(ql.Thirty360.European)
        fixed_rate = 0.03
        floating_leg_day_counter = ql.Actual360()
        start_date = todays_date + ql.Period(10 + random.randint(0, 359), ql.Days)

        # Floating leg
        floating_leg_frequency = ql.Semiannual
        euribor_index = ql.Euribor6M(forecasting_term_structure)
        spread = 0.0

        length_in_years = 2 + random.randint(0, 19)

        swap_type = ql.VanillaSwap.Payer

        maturity_date = start_date + ql.Period(length_in_years, ql.Years)
        fixed_schedule = ql.Schedule(start_date, maturity_date, ql.Period(fixed_leg_frequency), calendar,
                                     fixed_leg_convention, fixed_leg_convention, ql.DateGeneration.Forward, False)
        floating_schedule = ql.Schedule(start_date, maturity_date, ql.Period(floating_leg_frequency), calendar,
                                        floating_leg_convention, floating_leg_convention, ql.DateGeneration.Forward, False)

        random_vanilla_swap = ql.VanillaSwap(
            swap_type, nominal, fixed_schedule, fixed_rate, fixed_leg_day_counter,
            floating_schedule, euribor_index, spread, floating_leg_day_counter
        )

        swap_engine = ql.DiscountingSwapEngine(discounting_term_structure)
        random_vanilla_swap.setPricingEngine(swap_engine)

        portfolio.append((random_vanilla_swap, discounting_term_structure, forecasting_term_structure))

    return portfolio

In [3]:
def record_kernel(portfolio, dates):
    kernel = aadc.Kernel()
    kernel.start_recording()

    zero_rates = aadc.array(np.zeros(len(dates)))
    zero_args = zero_rates.mark_as_input()

    total_npv = aadc.idouble(0.0)

    log_linear_curve = ql.ZeroCurve(dates, zero_rates.tolist(), ql.Actual360(), ql.TARGET())

    for (swap, discounting_term_structure, forecasting_term_structure) in portfolio:
        discounting_term_structure.linkTo(log_linear_curve)
        forecasting_term_structure.linkTo(log_linear_curve)
        total_npv += swap.NPV()

    res = total_npv.mark_as_output()
    kernel.stop_recording()

    return (kernel, { res: zero_args })

In [4]:

num_trades = 1000
todays_date = ql.Date(12, ql.June, 2024)
calendar = ql.TARGET()

portfolio = createRandomSwaps(num_trades, todays_date, calendar)
dates = [todays_date + ql.Period(i, ql.Years) for i in range(1, 21)]

mark_time = time.time()
(kernel, request) = record_kernel(portfolio, dates)

print("Recording time: ", time.time() - mark_time)

kernel.print_passive_extract_locations()

You are using evaluation version of AADC. Expire date is 20240901
Recording time:  6.944329261779785
Number active to passive conversions: 0 while recording Python


In [5]:
def calc_risks(seed):
    random.seed(seed)
    zero_rates = np.zeros(len(dates))
    for i in range(len(dates)):
        zero_rates[i] = 0.0025 + 0.005 * 0.02 * random.randint(0, 99)

    args = list(request.values())[0]
    r = aadc.evaluate(kernel, request, { a: [x] for a, x in zip(args, zero_rates) }, aadc.ThreadPool(1))
    return (zero_rates, r)

calc_risks(42)

(array([0.0106, 0.0039, 0.0028, 0.0119, 0.006 , 0.0056, 0.0053, 0.0042,
        0.0119, 0.0038, 0.0111, 0.0119, 0.0094, 0.0036, 0.01  , 0.0079,
        0.0029, 0.0028, 0.0036, 0.0052]),
 ({Res(469483): array([2.01500209e+09])},
  {Res(469483): {Arg(7): array([-9.76468126e+08]),
    Arg(8): array([2.68719605e+09]),
    Arg(9): array([2.54149102e+09]),
    Arg(10): array([-6.17801339e+09]),
    Arg(11): array([-8.76170047e+09]),
    Arg(12): array([7.58053653e+09]),
    Arg(13): array([4.69419654e+09]),
    Arg(14): array([-1.11218181e+10]),
    Arg(15): array([-8.82925663e+08]),
    Arg(16): array([-4.69551855e+09]),
    Arg(17): array([-7.86757459e+09]),
    Arg(18): array([-1.2513143e+10]),
    Arg(19): array([9.8414201e+09]),
    Arg(20): array([7.20969485e+09]),
    Arg(21): array([1.57518928e+10]),
    Arg(22): array([2.98576163e+09]),
    Arg(23): array([-1.40481582e+10]),
    Arg(24): array([2.1454651e+10]),
    Arg(25): array([-4.55882262e+10]),
    Arg(26): array([-6.50050414e+

In [7]:
def plot_risks(seed):
    (zero_rates, r) = calc_risks(seed)

    risks = [item for subdict in r[1].values() for sublist in subdict.values() for item in sublist]
    assert(len(risks) == len(dates))

    plot_years = range(1, 21)
    assert(len(plot_years) == len(dates))

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(plot_years, zero_rates, marker='o', linestyle='-', color='b')
    plt.xlabel('Year')
    plt.ylabel('Zero Rate')
    plt.title('Zero Rate Curve')
    plt.grid(True)
    plt.ylim(0, max(zero_rates) * 1.5)

    # Plotting the riks
    plt.subplot(1, 2, 2)
    plt.plot(plot_years, risks, marker='o', linestyle='-', color='r')
    plt.xlabel('Year')
    plt.ylabel('Zero risk')
    plt.title('Zero risks')
    plt.grid(True)
    plt.ylim(min(risks) * 1.5, max(risks) * 1.5)

    plt.tight_layout()
    plt.show()


# Create interactive sliders for level and slope
seed_slider = widgets.FloatSlider(
    value=42,
    min=1,
    max=100,
    step=1,
    description='Seed:',
    continuous_update=False
)

# Use interactive function to update the plot
interactive_plot = interactive(plot_risks, seed=seed_slider)
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot


interactive(children=(FloatSlider(value=42.0, continuous_update=False, description='Seed:', min=1.0, step=1.0)…